In [ ]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
import operator
import os

print("✅ All imports successful")

In [ ]:
from typing import TypedDict, List, Optional

class HybridAgentState(TypedDict):
    # Core task
    task: str

    # -------- Plan-Execute --------
    plan: List[str]
    current_step: int
    step_results: List[str]

    # -------- Generation --------
    draft: str

    # -------- Reflection --------
    critique: str
    iterations: int

    # -------- Final --------
    final_output: str


In [ ]:
def planner(state: HybridAgentState) -> dict:
    prompt = f"""
    Break the following task into clear, ordered steps.

    Task:
    {state['task']}

    Return the steps as a numbered list.
    """

    print("\n🧠 PLANNER: Creating plan...")
    response = llm.invoke([HumanMessage(content=prompt)])

    plan = [
        line.strip("0123456789. ")
        for line in response.content.splitlines()
        if line.strip()
    ]

    print("📋 PLAN:")
    for i, step in enumerate(plan, 1):
        print(f"  Step {i}: {step}")

    return {
        "plan": plan,
        "current_step": 0,
        "step_results": []
    }


In [ ]:
def executor(state: HybridAgentState) -> dict:
    step_idx = state["current_step"]
    step = state["plan"][step_idx]

    prompt = f"""
    Execute the following step carefully and return the result.

    Step:
    {step}
    """

    print(f"\n⚙️ EXECUTOR: Running step {step_idx + 1}")
    response = llm.invoke([HumanMessage(content=prompt)])

    result = response.content

    print(f"✅ Result of step {step_idx + 1}:\n{result[:150]}...\n")

    return {
        "current_step": step_idx + 1,
        "step_results": state["step_results"] + [result]
    }


In [ ]:
from typing import Literal

def should_continue_execution(
    state: HybridAgentState
) -> Literal["executor", "generator"]:

    if state["current_step"] < len(state["plan"]):
        return "executor"

    print("🏁 All execution steps completed\n")
    return "generator"


In [ ]:
def generator(state: HybridAgentState) -> dict:
    combined_results = "\n\n".join(state["step_results"])

    prompt = f"""
    Using the information below, create a complete response to the task.

    Task:
    {state['task']}

    Collected Information:
    {combined_results}

    Create a clear and coherent draft.
    """

    print("✍️ GENERATOR: Creating initial draft...")
    response = llm.invoke([HumanMessage(content=prompt)])

    print("\n📝 INITIAL DRAFT:\n")
    print(response.content)

    return {"draft": response.content}


In [ ]:
def critic(state: HybridAgentState) -> dict:
    prompt = f"""
    Critique the following response.

    Task:
    {state['task']}

    Response:
    {state['draft']}

    Focus on:
    - Beginner-friendliness
    - Clarity
    - Completeness

    If excellent, say "APPROVED: reason".
    Otherwise, explain what needs improvement.
    """

    print("\n🔍 CRITIC: Evaluating draft...")
    response = llm.invoke([HumanMessage(content=prompt)])

    print("\n🧪 CRITIQUE:\n")
    print(response.content)

    return {
        "critique": response.content,
        "iterations": state["iterations"] + 1
    }


In [ ]:
MAX_REFLECTIONS = 2

def refiner(state: HybridAgentState) -> dict:
    prompt = f"""
    Improve the response based on the critique.

    Task:
    {state['task']}

    Current Draft:
    {state['draft']}

    Critique:
    {state['critique']}

    Produce a refined, beginner-friendly version.
    """

    print(f"\n🔁 REFINER: Refining (iteration {state['iterations']})...")
    response = llm.invoke([HumanMessage(content=prompt)])

    print("\n✨ REFINED DRAFT:\n")
    print(response.content)

    return {"draft": response.content}


In [ ]:
def should_refine_again(
    state: HybridAgentState
) -> Literal["refiner", "finalizer"]:

    if "APPROVED" in state["critique"].upper():
        print("✅ Draft approved")
        return "finalizer"

    if state["iterations"] >= MAX_REFLECTIONS:
        print("⚠️ Max reflection iterations reached")
        return "finalizer"

    return "refiner"


In [ ]:
def finalizer(state: HybridAgentState) -> dict:
    print("\n🎉 FINAL OUTPUT READY\n")
    return {"final_output": state["draft"]}


In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(HybridAgentState)

builder.add_node("planner", planner)
builder.add_node("executor", executor)
builder.add_node("generator", generator)
builder.add_node("critic", critic)
builder.add_node("refiner", refiner)
builder.add_node("finalizer", finalizer)

builder.add_edge(START, "planner")
builder.add_edge("planner", "executor")

builder.add_conditional_edges(
    "executor",
    should_continue_execution,
    {
        "executor": "executor",
        "generator": "generator"
    }
)

builder.add_edge("generator", "critic")

builder.add_conditional_edges(
    "critic",
    should_refine_again,
    {
        "refiner": "refiner",
        "finalizer": "finalizer"
    }
)

builder.add_edge("refiner", "critic")
builder.add_edge("finalizer", END)

hybrid_agent = builder.compile()


In [ ]:
result = hybrid_agent.invoke({
    "task": "Research the benefits of Python programming, create a summary, and make it beginner-friendly",
    "plan": [],
    "current_step": 0,
    "step_results": [],
    "draft": "",
    "critique": "",
    "iterations": 0,
    "final_output": ""
})
